In [3]:
import sys
from pathlib import Path

# Add the parent directory to Python path to import starter module
sys.path.insert(0, str(Path().resolve().parent))

from dotenv import load_dotenv
from starter.testing_utils import load_udahub_table

import asyncio

load_dotenv()

True

In [5]:
import pandas as pd 

In [12]:
pd.read_sql_query(f"SELECT * FROM users", con="sqlite:///./data/core/udahub.db")

,user_id,account_id,external_user_id,user_name,created_at,updated_at
0,662dfd3a-b000-49c1-b4c2-5259a3558daf,cultpass,a4ab87,Alice Kingsley,2026-02-07 02:23:40,2026-02-07 02:23:40
